# Лабораторная работа №5 Белоцкий Дмитрий

In [2]:
import numpy as np
import os
import pandas
from scipy.misc import derivative

# Задание 1.

In [3]:
def f_0(x):
    return np.exp(2 * x) * np.cos(3 * x)


def f_1(x):
    return np.sin(np.log(x))


def f_2(x):
    return np.log(x)


def f_3(x):
    return np.sin(x) + np.cos(x)


def one_way_difference_method(func, x, n=1, dx = 1e-3):
    if n == 1:
        return (func(x + dx) - func(x)) / dx
    else:
        return (one_way_difference_method(func, x + dx, n - 1) - one_way_difference_method(func, x, n - 1)) / dx


def two_way_difference_method(func, x, n=1, dx=1e-3):
    if n == 1:
        return (func(x + dx) - func(x - dx)) / (2*dx)
    else:
        return (two_way_difference_method(func, x + dx, n - 1) - two_way_difference_method(func, x - dx,
                                                                                             n - 1)) / (2*dx)


def left_difference_method(func, x, n=1, dx=1e-3):
    if n == 1:
        return (func(x) - func(x - dx)) / dx
    else:
        return (left_difference_method(func, x, n - 1) - left_difference_method(func, x - dx, n - 1)) / dx

In [26]:
funcs = [f_0, f_1, f_2, f_3]
points_x = np.array([[0, 0.3, 0.6], [2.0, 2.4, 2.6], [1, 1.1, 1.3, 1.4], [0, 0.25, 0.5, 1]])
input_x = [0.25, 0.51, 0.99, 1.09, 1.89, 2.39]
max_err_left, max_err_two, max_err_right = [], [], [] 
for n in range(0, len(funcs)):
    print('     Функция ' + str(n + 1) + ":")
    for i in range(6):
        print(str((i+2)//2) + '-ая производная в точке ' + str(input_x[i]) + ': ')
        real = derivative(funcs[n], input_x[i], dx=1e-3, n=(i+2)//2, order=5)
        left = left_difference_method(funcs[n], input_x[i], (i+2)//2)
        both = two_way_difference_method(funcs[n], input_x[i], (i+2)//2)
        right = one_way_difference_method(funcs[n], input_x[i], (i+2)//2)
        print('Реальное значение:            ' + str(real))
        print('Метод левосторонней разности:  ' + str(left))
        max_err_left.append(abs((left - real)/real))
        print('Метод двусторонней разности:  ' + str(both))
        max_err_two.append(abs((both - real)/real))
        print('Метод правосторонней разности: ' + str(right))
        max_err_right.append(abs((right - real)/real))
    mean_left, mean_both, mean_right = 0, 0, 0
    for i in range(len(max_err_left)):
        mean_left += max_err_left[i]
        mean_both += max_err_two[i]
        mean_right += max_err_right[i]
    mean_left /= len(max_err_left)
    mean_both /= len(max_err_left)
    mean_right /= len(max_err_left)
        
print('\nСредняя ошибка для методов')
print('Метод левосторонней разности   :  ' + str(mean_left))
print('Метод двусторонней разности    :  ' + str(mean_both))
print('Метод правосторонней разности  :  ' + str(mean_right))

     Функция 1:
1-ая производная в точке 0.25: 
Реальное значение:            -0.95879496448599
Метод левосторонней разности:  -0.9490470270894225
Метод двусторонней разности:  -0.9588058988915238
Метод правосторонней разности: -0.9685647706936251
1-ая производная в точке 0.51: 
Реальное значение:            -8.086452383251839
Метод левосторонней разности:  -8.069549334109762
Метод двусторонней разности:  -8.086457406650066
Метод правосторонней разности: -8.10336547919037
2-ая производная в точке 0.99: 
Реальное значение:            20.841343442490334
Метод левосторонней разности:  20.52478250824663
Метод двусторонней разности:  20.841675984595298
Метод правосторонней разности: 21.159068277043502
2-ая производная в точке 1.09: 
Реальное значение:            57.46111739712845
Метод левосторонней разности:  57.04787238514086
Метод двусторонней разности:  57.46142009854793
Метод правосторонней разности: 57.87542186652672
3-ая производная в точке 1.89: 
Реальное значение:            -1421.

Метод двусторонней разности оказался самым точным. Методы левосторонней и правосторонней разности имеют примерно одниковую
относительную ошибку

# Задание 2.

In [5]:
def spline_interpolation(x, points_x, points_y):
    n = len(points_x)
    h = np.diff(points_x)
    df = np.diff(points_y)
    A = np.zeros((n, n))
    B = np.zeros((n, ))
    A[0, 0] = 1
    A[n-1, n-1] = 1
    for i in range(1, n-1):
        A[i, i - 1] = h[i-1]
        A[i][i] = 2 * (h[i-1] + h[i])
        A[i, i + 1] = h[i]
        B[i] = 3 * (df[i] / h[i] - df[i - 1] / h[i - 1])
    c = np.linalg.solve(A, B)
    d = np.zeros((n - 1,))
    b = np.zeros((n - 1,))
    for i in range(0, len(d)):
        d[i] = (c[i + 1] - c[i]) / (3 * h[i])
        b[i] = (df[i] / h[i]) - (h[i] / 3) * (2 * c[i] + c[i + 1])
    if x <= points_x[0]:
        return points_y[0] + b[0] * (x - points_x[0]) + c[0] * (x - points_x[0]) ** 2 + d[0] * (
                    x - points_x[0]) ** 3
    for i in range(0, n-1):
        if points_x[i] <= x <= points_x[i+1]:
            return points_y[i] + b[i]*(x - points_x[i]) + c[i] * (x - points_x[i])**2+d[i]*(x - points_x[i])**3
    return points_y[n-2] + b[n-2]*(x - points_x[n-2]) + c[n-2] * (x - points_x[n-2])**2+d[n-2]*(x - points_x[n-2])**3


def MNK_interpolation(x, points_x, points_y):
    n = len(points_x)
    k = 5
    X = np.ones((n, k))
    y = np.array(points_y)
    for i in range(n):
        for j in range(k):
            X[i][j] = points_x[i]**j
    A = np.dot(X.transpose(), X)
    y = np.dot(X.transpose(), y)
    a = np.linalg.solve(A, y)
    ans = 0
    for i in range(k):
        ans += a[i] * x**i
    return ans

In [15]:
def mid_point_method(func):
    integral = 0
    x = np.linspace(days[0], days[-1], 100)
    mids = np.linspace(days[0], days[-1], 200)
    step = (days[-1]-days[0]) / 100
    y = [func(mids[i], days, cases) for i in range(1, 200, 2)]
    for i in y:
        integral += i * step
    return integral


def trapeze_method(func):
    integral = 0
    x = np.linspace(days[0], days[-1], 100)
    step = (days[-1]-days[0]) / 100
    y = [func(i, days, cases) for i in x]
    for i in range(99):
        integral += (y[i]+y[i+1])*step/2
    return integral

In [16]:
files = os.listdir('./covid_stat/csse_covid_19_daily_reports')
cases = []
for file in files:
    f = pandas.read_csv('./covid_stat/csse_covid_19_daily_reports/' + file)
    cases.append(np.sum(f['Confirmed']))
days = np.array(range(len(cases)))+1

total_cases_num = 0
for num in cases:
    total_cases_num += num
int1 = mid_point_method(spline_interpolation)
int2 = mid_point_method(MNK_interpolation)
int3 = trapeze_method(spline_interpolation)
int4 = trapeze_method(MNK_interpolation)
print('Всего зараженных:                  ' + str(total_cases_num))
print('Интеграл от приближения сплайнами: ')
print('   Метод средней тчоки:    '+ str(int1) + ' (относительная погрешность: ' + str((total_cases_num-int1)/total_cases_num) + ')')
print('   Метод трапеций:         '+ str(int3) + ' (относительная погрешность: ' + str((total_cases_num-int3)/total_cases_num) + ')')
print('Интеграл от приближения МНК: ')
print('   Метод средней точки:    '+ str(int2) + ' (относительная погрешность: ' + str((total_cases_num-int2)/total_cases_num) + ')')
print('   Метод трапеций:         '+ str(int4) + ' (относительная погрешность: ' + str((total_cases_num-int4)/total_cases_num) + ')')

Всего зараженных:                  115373662.0
Интеграл от приближения сплайнами: 
   Метод средней тчоки:    115017232.8969703 (относительная погрешность: 0.003089345495765733)
   Метод трапеций:         112153227.78601487 (относительная погрешность: 0.027913079624577838)
Интеграл от приближения МНК: 
   Метод средней точки:    115096350.89282545 (относительная погрешность: 0.002403591100146804)
   Метод трапеций:         112244671.60241269 (относительная погрешность: 0.027120491309249716)


Остаточный член в методе трапеций имеет вид: f'(c)*(b-a)^3 / 12 , а в методе средней точки f'(c)*(b-a)^3 / 24. Хоть и значения 
    погрешности в нашем примере отличаются на порядок, остаточный член дает лишь верхнюю оценку для погрешности, и нам просто
    "повезло" с методом средней точки. Оба метода асимптотически имеют кубическую сходимость